## Intermediate Parallel Computing

### Part 4 of 4

### PySpark SQL II: Let’s Go Spatial!

### In this segment we will learn:
* Spark SQL and Geopandas.
* Querying spatial data with PySpark SQL.

## Reminder
<a href="#/slide-2-0" class="navigate-right" style="background-color:blue;color:white;padding:8px;margin:2px;font-weight:bold;">Continue with the lesson</a>

<br>
</br>
<font size="+1">

By continuing with this lesson you are granting your permission to take part in this research study for the Hour of Cyberinfrastructure: Developing Cyber Literacy for GIScience project. In this study, you will be learning about cyberinfrastructure and related concepts using a web-based platform that will take approximately one hour per lesson. Participation in this study is voluntary.

Participants in this research must be 18 years or older. If you are under the age of 18 then please exit this webpage or navigate to another website such as the Hour of Code at https://hourofcode.com, which is designed for K-12 students.

If you are not interested in participating please exit the browser or navigate to this website: http://www.umn.edu. Your participation is voluntary and you are free to stop the lesson at any time.

For the full description please navigate to this website: <a href="../../gateway-lesson/gateway/gateway-1.ipynb">Gateway Lesson Research Study Permission</a>.

</font>

In [ ]:
# This code cell starts the necessary setup for Hour of CI lesson notebooks.
# First, it enables users to hide and unhide code by producing a 'Toggle raw code' button below.
# Second, it imports the hourofci package, which is necessary for lessons and interactive Jupyter Widgets.
# Third, it helps hide/control other aspects of Jupyter Notebooks to improve the user experience
# This is an initialization cell
# It is not displayed because the Slide Type is 'Skip'

from IPython.display import HTML, IFrame, Javascript, display
from ipywidgets import interactive
import ipywidgets as widgets
from ipywidgets import Layout

import getpass # This library allows us to get the username (User agent string)

# import package for hourofci project
import sys
sys.path.append('../../supplementary') # relative path (may change depending on the location of the lesson notebook)
import hourofci

# load javascript to initialize/hide cells, get user agent string, and hide output indicator
# hide code by introducing a toggle button "Toggle raw code"
HTML(''' 
    <script type="text/javascript" src=\"../../supplementary/js/custom.js\"></script>
    
    <style>
        .output_prompt{opacity:0;}
    </style>
    
    <input id="toggle_code" type="button" value="Toggle raw code">
''')

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
conf = SparkConf().setAppName("hourofci").setMaster("local[4]")

spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext
sc

In [ ]:
from IPython import get_ipython
import osmnx as ox 
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
get_ipython().run_line_magic('matplotlib', 'inline')
#Tells jupyter to plot matplotlib figures inline



In [ ]:
place = 'St Paul, MN'

tags = {'amenity':'cafe', 'cuisine':'coffee-shop'}  
coffee_shops = ox.geometries_from_place(place, tags) 

# Convert to UTM
coffee_shops = coffee_shops.to_crs('epsg:3174')[['name', 'geometry']]

# coffee_shops.info()
coffee_shops.head()


In [ ]:
coffee_shops.info()

In [ ]:
coffee_shops.plot()

In [ ]:
minneapolis = ox.geocode_to_gdf('Minneapolis, MN')
minneapolis = minneapolis.to_crs('epsg:3174')
minneapolis.plot()

In [ ]:
counties = ox.geocode_to_gdf(['Hennepin County', 'Ramsey County'])
counties = counties.to_crs('epsg:3174')
counties.plot()


In [ ]:
# gdf.plot(figsize = (13, 15), column = 'pop_est', cmap = 'Dark2')

base_map = counties.plot(figsize = (10,10))

minneapolis.plot(ax = base_map, color = 'red')



In [ ]:
counties['wkt'] = pd.Series(map(lambda geom: str(geom.wkt), counties['geometry']), index=counties.index, dtype='string')

minneapolis['wkt'] = pd.Series(map(lambda geom: str(geom.wkt), minneapolis['geometry']), index=minneapolis.index, dtype='string')




In [ ]:

minneapolis.drop('geometry', axis=1, inplace=True)
minneapolis_df = spark.createDataFrame(minneapolis).cache()
minneapolis_df



In [ ]:
counties.drop('geometry', axis=1, inplace=True)

# world_spark_df = spark.createDataFrame(drop_geometry_column(world)).cache()
counties_df = spark.createDataFrame(counties).cache()
counties_df


In [ ]:
counties_df.printSchema()

In [ ]:
minneapolis_df.printSchema()

In [ ]:
counties_df.createOrReplaceTempView("counties_table")
minneapolis_df.createOrReplaceTempView("minneapolis_table")

In [ ]:
cnt = spark.sql(
            """
            SELECT display_name, wkt as geometry, importance, type, class
            FROM counties_table
            """)

In [ ]:
# cnt.show()

In [ ]:


intsec = spark.sql(
            """
            SELECT display_name, wkt as geometry, importance, type, class
            FROM counties_table c, minneapolis_table m
            WHERE ST_Intersects(c.geometry, m.geometry)
            """)

In [ ]:

# This code cell loads the Interact Textbox that will ask users for their name
# Once they click "Create Certificate" then it will add their name to the certificate template
# And present them a PDF certificate
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw

from ipywidgets import interact

def make_cert(learner_name, lesson_name):
    cert_filename = 'hourofci_certificate.pdf'

    img = Image.open("../../supplementary/hci-certificate-template.jpg")
    draw = ImageDraw.Draw(img)

    cert_font   = ImageFont.truetype('../../supplementary/cruft.ttf', 150)
    cert_fontsm = ImageFont.truetype('../../supplementary/cruft.ttf', 80)
    
    _,_,w,h = cert_font.getbbox(learner_name)  
    draw.text( xy = (1650-w/2,1100-h/2), text = learner_name, fill=(0,0,0),font=cert_font)
    
    _,_,w,h = cert_fontsm.getbbox(lesson_name)
    draw.text( xy = (1650-w/2,1100-h/2 + 750), text = lesson_name, fill=(0,0,0),font=cert_fontsm)
    
    img.save(cert_filename, "PDF", resolution=100.0)   
    return cert_filename


interact_cert=interact.options(manual=True, manual_name="Create Certificate")

@interact_cert(name="Your Name")
def f(name):
    print("Congratulations",name)
    filename = make_cert(name, 'Intermediate Parallel Computing')
    print("Download your certificate by clicking the link below.")
    
    


<font size="+1"><a style="background-color:blue;color:white;padding:12px;margin:10px;font-weight:bold;" href="hourofci_certificate.pdf?download=1" download="hourofci_certificate.pdf">Download your certificate</a></font>
